<a href="https://colab.research.google.com/github/confusedolive/Jeronimo-CE888/blob/main/lab04/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
np.set_printoptions(precision=3)

In [ ]:
jester_data = pd.read_csv(r'https://raw.githubusercontent.com/albanda/CE888/master/lab4-recommender/jester-data-1.csv', na_values=99)
jester_df = jester_data.copy()

'''number of users is 24983 , 100 jokes , values to be treated as
nans are 99, column 74 to be dropped as its the amount of jokes rated by user'''

#drop column which contains number of jokes rated by user
jester_df.drop(labels='74',inplace=True, axis=1)
#print(jester_df)

#best and worst rated jokes
best_rating_joke = jester_df.mean().idxmax()
worst_rating_joke = jester_df.mean().idxmin()

print(jester_df[worst_rating_joke].mean())
print(jester_df.iloc[2143, 54])

-3.833520447530859
2.18


In [ ]:
#Helper function
''' Label 10% of the dataset cells that are not Nans as 99, validation set
Keep actual values of the cells so you can use them later'''

arr = jester_df.values
rated = np.where(~np.isnan(arr))


def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(~np.isnan(arr))
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  
  return new_data, (rated[0][idx], rated[1][idx])


new_arr, idx = replace(arr)
print(new_arr)
print(rated)

[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [  nan   nan   nan ...   nan   nan   nan]
 [  nan  8.35   nan ...   nan   nan   nan]
 ...
 [  nan   nan   nan ...   nan   nan   nan]
 [  nan   nan   nan ...   nan   nan   nan]
 [ 2.43 99.   -3.98 ...   nan   nan   nan]]
(array([    0,     0,     0, ..., 24981, 24981, 24981]), array([ 0,  1,  2, ..., 69, 70, 86]))


In [ ]:

user_ratings = new_arr
arr = jester_df.values

latent_user_preferences = np.random.random((user_ratings.shape[0], 5))
latent_item_features = np.random.random((user_ratings.shape[1], 5))


def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0004):
    #print(item_id)
    predicted_rating = predict_rating(user_id, item_id)
    err =  predicted_rating - rating
    #print(err)
    user_pref_values = latent_user_preferences[user_id]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err
    

def sgd(iterations):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    mse_history = []
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = arr[user_id, item_id]
                if not np.isnan(rating):
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        #if (iteration % 10000) == 0:
        print('Iteration %d/%d:\tMSE=%.6f' % (iteration, iterations, mse))
        mse_history.append(mse)
    return mse_history


# values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
# comparison_data = pd.DataFrame(values)
# print(comparison_data)
sgd(10)


Iteration 0/10:	MSE=25.026114
Iteration 1/10:	MSE=24.498123
Iteration 2/10:	MSE=24.070436
Iteration 3/10:	MSE=23.575935
Iteration 4/10:	MSE=22.801752
Iteration 5/10:	MSE=21.263625
Iteration 6/10:	MSE=19.309714
Iteration 7/10:	MSE=18.200498
Iteration 8/10:	MSE=17.791976
Iteration 9/10:	MSE=17.606034


[25.02611432096978,
 24.49812349570876,
 24.07043564366498,
 23.575935492205026,
 22.80175151666691,
 21.263624988162235,
 19.309713538913773,
 18.200498375671053,
 17.791975546003982,
 17.606034354655712]

In [ ]:
movies_items = pd.read_excel('movies_latent_factors.xlsx',sheet_name = 'Items', index_col='Movie ID')
movie_users = pd.read_excel('movies_latent_factors.xlsx', sheet_name = 'Users', index_col='User')

#print(movies_items.loc[movies_items['Factor3'].idxmax()])
#print(movies_items.loc[122])

user_rating = movie_users.loc[5338].values
movie_rating = movies_items.loc[36657].values[1:]
print(user_rating.dot(movie_rating))

val = movie_users.loc[3878].values
test = {}
for x in movies_items.values:
  test[x[0]] = val.dot(x[1:])
  

sort_dict= dict(sorted(test.items(), key=lambda item: item[1])) 
c = list(sort_dict.keys())
for x in c[-2:]:
  print(f'{x} {movies_items.loc[movies_items["Title"] == x].index[0]}')
print(sort_dict)

-0.2280152087189911
Dumb & Dumber (1994) 8467
Back to the Future (1985) 105
{'The Lord of the Rings: The Two Towers (2002)': -2.0611500084617242, 'The Lord of the Rings: The Fellowship of the Ring (2001)': -1.9924328167852676, 'The Lord of the Rings: The Return of the King (2003)': -1.9801680405720616, 'The Matrix Reloaded (2003)': -1.7920124797821286, "Charlie's Angels (2000)": -1.2345129694860018, 'Batman Forever (1995)': -1.0779532025164928, 'Spider-Man 2 (2004)': -1.0023885156591041, 'Star Wars: Episode II - Attack of the Clones (2002)': -0.966949811337707, 'Mission: Impossible II (2000)': -0.8003154498975374, 'Pirates of the Caribbean: The Curse of the Black Pearl (2003)': -0.6860519680580948, 'Spider-Man (2002)': -0.6804796103520693, 'Crouching Tiger Hidden Dragon (Wo hu cang long) (2000)': -0.5593304269773673, 'X2: X-Men United (2003)': -0.5556356202530627, 'Shrek 2 (2004)': -0.4305716435120944, 'Twister (1996)': -0.38504843791260945, 'X-Men (2000)': -0.3762269856652471, 'The Pa